In [1]:
# Install required packages
!pip install tqdm
!apt-get update
!apt-get install -y wget
!pip install selenium
!apt-get install -y chromium-browser
!apt-get install -y chromium-chromedriver

# Import necessary libraries
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup  # For parsing HTML content
from urllib.parse import urljoin, urlparse  # For handling URLs
import urllib.request  # For making HTTP requests
import time  # For handling time-related operations
import os  # For interacting with the operating system (relate to dir, folder, file)
from tqdm import tqdm  # For displaying progress bars (visualize progress)
import concurrent.futures  # For multi-threading
import json  # For writing to a text file
from PIL import Image  # For handling images

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Ign:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,449 kB]
Get:14 https://ppa.launchpad

In [2]:
class UrlScraper:
    # Constructor
    def __init__(self, url_template, max_images=50, max_workers=4):
        self.url_template = url_template  # Link crawl
        self.max_images = max_images  # Max images
        self.max_workers = max_workers  # Thread
        self.setup_environment()  # Call for set up environment

    # Set up environment for selenium
    def setup_environment(self):
        os.environ['PATH'] += ':/usr/lib/chromium-browser/'
        os.environ['PATH'] += ':/usr/lib/chromium-browser/chromedriver/'

    def get_url_images(self, term):
        """
        Crawl the URLs of images by term

        Parameters:
        term (str): The name of the animal, plant, scenery, or furniture

        Returns:
        urls (list): List of URLs of images
        """

        # Initialize Chrome driver
        options = webdriver.ChromeOptions()
        options.add_argument('--headless')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        driver = webdriver.Chrome(options=options)

        url = self.url_template.format(search_term=term)
        driver.get(url)

        # Start crawling URLs of images
        urls = []
        more_content_available = True

        pbar = tqdm(total=self.max_images, desc=f"Fetching images for {term}")  # Set up for visualizing progress

        while len(urls) < self.max_images and more_content_available:
            soup = BeautifulSoup(driver.page_source, "html.parser")
            img_tags = soup.find_all("img")

            for img in img_tags:
                if len(urls) >= self.max_images:
                    break
                if 'src' in img.attrs:
                    href = img.attrs['src']
                    img_path = urljoin(url, href)
                    img_path = img_path.replace("_m.jpg", "_b.jpg").replace("_n.jpg", "_b.jpg").replace("_w.jpg", "_b.jpg")
                    if img_path == "https://combo.staticflickr.com/ap/build/images/getty/IStock_corporate_logo.svg":
                        continue
                    urls.append(img_path)
                    pbar.update(1)

            try:
                load_more_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, '//button[@id="yui_3_16_0_1_1721642285931_28620"]'))
                )
                load_more_button.click()
                time.sleep(2)
            except:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(2)

            new_soup = BeautifulSoup(driver.page_source, "html.parser")
            new_img_tags = new_soup.find_all("img", loading_="lazy")
            if len(new_img_tags) == len(img_tags):
                more_content_available = False
            img_tags = new_img_tags

        pbar.close()
        driver.quit()
        return urls

    def scrape_urls(self, categories):
        """
        Call the get_url_images method to retrieve all URLs of objects in categories

        Parameters:
        categories (dict): Dictionary of all objects to collect images with format
                           categories{"name_object": [value1, value2, ...]}

        Returns:
        all_urls (dict): Dictionary containing image URLs
        """
        all_urls = {category: {} for category in categories}

        # Use multithreading for efficiency
        with concurrent.futures.ThreadPoolExecutor(max_workers=self.max_workers) as executor:
            future_to_term = {executor.submit(self.get_url_images, term): (category, term)
                              for category, terms in categories.items() for term in terms}

            for future in tqdm(concurrent.futures.as_completed(future_to_term), total=len(future_to_term), desc="Overall Progress"):
                category, term = future_to_term[future]
                try:
                    urls = future.result()
                    all_urls[category][term] = urls
                    print(f"\nNumber of images retrieved for {term}: {len(urls)}")
                except Exception as exc:
                    print(f"\n{term} generated an exception: {exc}")
        return all_urls

    def save_to_file(self, data, filename):
        """
        Save data to a JSON file.

        Parameters:
        data (dict): Data to be saved.
        filename (str): Name of the JSON file.

        Returns:
        None
        """
        with open(filename, 'w') as file:
            json.dump(data, file, indent=4)
        print(f"Data saved to {filename}")


In [3]:
categories = {
    "animal": ["Monkey", "Elephant", "cows", "Cat", "Dog", "bear", "fox", "Civet", "Pangolins", "Rabbit", "Bats", "Whale", "Cock", "Owl", "flamingo", "Lizard", "Turtle", "Snake", "Frog", "Fish", "shrimp", "Crab", "Snail", "Coral", "Jellyfish", "Butterfly", "Flies", "Mosquito", "Ants", "Cockroaches", "Spider", "scorpion", "tiger", "bird", "horse", "pig", "Alligator", "Alpaca", "Anteater", "donkey", "Bee", "Buffalo", "Camel", "Caterpillar", "Cheetah", "Chicken", "Dragonfly", "Duck", "panda", "Giraffe"],
    "plant": ["Bamboo", "Apple", "Apricot", "Banana", "Bean", "Wildflower", "Flower", "Mushroom", "Weed", "Fern", "Reed", "Shrub", "Moss", "Grass", "Palmtree", "Corn", "Tulip", "Rose", "Clove", "Dogwood", "Durian", "Ferns", "Fig", "Flax", "Frangipani", "Lantana", "Hibiscus", "Bougainvillea", "Pea", "OrchidTree", "RangoonCreeper", "Jackfruit", "Cottonplant", "Corneliantree", "Coffeeplant", "Coconut", "wheat", "watermelon", "radish", "carrot"],
    "furniture": ["bed", "cabinet", "chair", "chests", "clock", "desks", "table", "Piano", "Bookcase", "Umbrella", "Clothes", "cart", "sofa", "ball", "spoon", "Bowl", "fridge", "pan", "book"],
    "scenery": ["Cliff", "Bay", "Coast", "Mountains", "Forests", "Waterbodies", "Lake", "desert", "farmland", "river", "hedges", "plain", "sky", "cave", "cloud", "flowergarden", "glacier", "grassland", "horizon", "lighthouse", "plateau", "savannah", "valley", "volcano", "waterfall"]
}

urltopic = {"flickr": "https://www.flickr.com/search/?text={search_term}"}
scraper = UrlScraper(url_template=urltopic["flickr"], max_images=2, max_workers=5)
image_urls = scraper.scrape_urls(categories)
scraper.save_to_file(image_urls, 'image_urls.json')

Fetching images for Monkey:  50%|█████     | 1/2 [00:03<00:03,  3.45s/it]

Fetching images for Dog:   0%|          | 0/2 [00:00<?, ?it/s]


Fetching images for cows:   0%|          | 0/2 [00:00<?, ?it/s]

Fetching images for Dog:  50%|█████     | 1/2 [00:00<00:00,  1.08it/s]


Overall Progress:   1%|          | 1/134 [01:00<2:14:26, 60.65s/it]


Number of images retrieved for Monkey: 2



Overall Progress:   1%|▏         | 2/134 [01:06<1:02:24, 28.37s/it]


Number of images retrieved for Dog: 2


Overall Progress:   2%|▏         | 3/134 [01:06<34:13, 15.67s/it]  

...


Number of images retrieved for plateau: 2



Fetching images for volcano:   0%|          | 0/2 [00:00<?, ?it/s]

Fetching images for waterfall:   0%|          | 0/2 [00:00<?, ?it/s]

Fetching images for volcano: 100%|██████████| 2/2 [00:14<00:00,  7.14s/it]



Number of images retrieved for waterfall: 2


Overall Progress: 100%|██████████| 134/134 [15:47<00:00,  7.07s/it]


Number of images retrieved for volcano: 2
Data saved to image_urls.json


In [4]:

class ImageDownloader:
    def __init__(self, json_file, download_dir='Dataset', max_workers=4, delay=1):
        self.json_file = json_file  # File chứa URL của ảnh dưới dạng JSON
        self.download_dir = download_dir  # Tên thư mục để lưu trữ ảnh
        self.max_workers = max_workers  # Số lượng luồng
        self.delay = delay  # Độ trễ lịch sự
        self.filename = set()  # Để lưu trữ các đường dẫn tên file
        self.setup_directory()  # Thiết lập cấu trúc thư mục

    def setup_directory(self):
        if not os.path.exists(self.download_dir):
            os.makedirs(self.download_dir)

    def read_json(self):
        """
        Đọc file JSON và trả về dữ liệu.

        Trả về:
        data (dict): Dữ liệu đọc từ file JSON.
        """
        with open(self.json_file, 'r') as file:
            data = json.load(file)
        return data

    def is_valid_url(self, url):
        """
        Kiểm tra xem URL có hợp lệ không.

        Tham số:
        url (str): URL cần kiểm tra.

        Trả về:
        bool: True nếu URL hợp lệ, False nếu không.
        """
        try:
            with urllib.request.urlopen(url) as response:
                if response.status == 200 and 'image' in response.info().get_content_type():
                    return True
        except Exception:
            return False

    def download_image(self, url, category, term, pbar):
        """
        Tải ảnh từ URL đã cho.

        Tham số:
        url (str): URL của ảnh cần tải.
        category (str): Danh mục của ảnh.
        term (str): Từ khóa liên quan đến ảnh.
        pbar (tqdm): Đối tượng thanh tiến trình.

        Trả về:
        str: Thông báo chỉ ra trạng thái của việc tải.
        """
        if not self.is_valid_url(url):
            pbar.update(1)
            return f"Invalid URL: {url}"

        category_dir = os.path.join(self.download_dir, category)
        if not os.path.exists(category_dir):
            os.makedirs(category_dir)

        term_dir = os.path.join(category_dir, term)
        if not os.path.exists(term_dir):
            os.makedirs(term_dir)

        filename = os.path.join(term_dir, os.path.basename(urlparse(url).path))
        self.filename.add(filename)  # Ghi lại đường dẫn tên file

        try:
            urllib.request.urlretrieve(url, filename)
            pbar.update(1)
            return f"Downloaded: {url}"
        except Exception as e:
            pbar.update(1)
            return f"Failed to download {url}: {str(e)}"

    def download_images(self):
        """
        Tải ảnh từ file JSON.

        Trả về:
        None
        """
        data = self.read_json()
        download_tasks = []

        total_images = sum(len(urls) for terms in data.values() for urls in terms.values())
        with tqdm(total=total_images, desc="Downloading images") as pbar:
            with concurrent.futures.ThreadPoolExecutor(max_workers=self.max_workers) as executor:
                for category, terms in data.items():
                    for term, urls in terms.items():
                        for url in urls:
                            download_tasks.append(executor.submit(self.download_image, url, category, term, pbar))
                            time.sleep(self.delay)  # Độ trễ lịch sự

                for future in concurrent.futures.as_completed(download_tasks):
                    print(future.result())

        self.export_filename()

    def export_filename(self):
        """
        Xuất danh sách đường dẫn tên file vào một file text.

        Trả về:
        None
        """
        with open('filename.txt', 'w') as file:
            for filename in sorted(self.filename):
                file.write(f"{filename}\n")


In [5]:
downloader = ImageDownloader(json_file='image_urls.json', download_dir='Dataset', max_workers=4, delay=1)
downloader.download_images()
downloader.export_filename()

Downloaded: https://live.staticflickr.com/7259/7577390134_b1cda83627_b.jpg
Downloaded: https://live.staticflickr.com/4437/36367572521_a8d5235e47_b.jpg
Downloaded: https://live.staticflickr.com/7377/9641031199_ea7460e902_b.jpg
Downloaded: https://live.staticflickr.com/5596/14307815484_87d075481a_b.jpg
Downloaded: https://live.staticflickr.com/1962/44791540314_c8ae2861d2_b.jpg
Downloaded: https://live.staticflickr.com/65535/33659529142_2b6c3241eb_b.jpg
Downloaded: https://live.staticflickr.com/2815/9510353068_b0b4f552d3_b.jpg
Downloaded: https://live.staticflickr.com/4269/35172789496_a387d77f5f_b.jpg
Downloaded: https://live.staticflickr.com/6161/6177244431_80f72e56f0_b.jpg
Downloaded: https://live.staticflickr.com/5254/5412377706_d77700fc38_b.jpg
Downloaded: https://live.staticflickr.com/8147/7471104002_169c3077ba_b.jpg
Downloaded: https://live.staticflickr.com/1934/45431020202_88040ae942_b.jpg
Downloaded: https://live.staticflickr.com/3506/3194806745_1465ae0373_b.jpg
Downloaded: https:

Downloaded: https://live.staticflickr.com/7446/26509038664_a8730f7bf4_b.jpg


Downloaded: https://live.staticflickr.com/65535/52750533076_fdfc390fcb_b.jpg


Downloaded: https://live.staticflickr.com/3099/3194860899_640ce169ac_b.jpg
Downloaded: https://live.staticflickr.com/4141/4871150060_c7e5e5175e_b.jpg


Downloaded: https://live.staticflickr.com/1189/561670551_efa5f9ae72_b.jpg


...

Downloaded: https://live.staticflickr.com/2679/4384395986_fcd011d5e1_b.jpg
Downloaded: https://live.staticflickr.com/747/22288603600_ac3080cab3_b.jpg


Downloaded: https://live.staticflickr.com/4628/39700034572_76878a296d_b.jpg


Downloaded: https://live.staticflickr.com/65535/49629211101_d46f735055.jpg
Downloaded: https://live.staticflickr.com/1380/5153065721_d56cf6c561_b.jpg


Downloaded: https://live.staticflickr.com/1560/24635030261_def899db51_b.jpg
Downloaded: https://live.staticflickr.com/5258/5489755530_b0596ef230_b.jpg


Downloaded: https://live.staticflickr.com/65535/53671954433_c45be77a79_b.jpg


In [6]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive/')

# Function to check and preprocess images
def check_and_preprocess_images(image_dir):
    """
    Check and preprocess images in the specified directory.

    Parameters:
    image_dir (str): The directory containing the images to be checked and preprocessed.

    Returns:
    None
    """
    for root, _, files in os.walk(image_dir):
        for file in files:
            file_path = os.path.join(root, file)
            try:
                with Image.open(file_path) as img:
                    # Check if image is smaller than 50x50 pixels
                    if img.size[0] < 50 or img.size[1] < 50:
                        os.remove(file_path)
                        print(f"Deleted {file_path}: Image too small ({img.size[0]}x{img.size[1]})")
                        continue

                    # Convert non-RGB images to RGB
                    if img.mode != 'RGB':
                        img = img.convert('RGB')
                        img.save(file_path)
                        print(f"Converted {file_path} to RGB")

            except Exception as e:
                # If file is not an image, delete it
                os.remove(file_path)
                print(f"Deleted {file_path}: Not an image or corrupted file ({str(e)})")

# Call the function to check and preprocess images in the 'Dataset' directory
check_and_preprocess_images('Dataset')

# Zip the processed dataset and save it to Google Drive
!zip -r /content/drive/MyDrive/Clean_Dataset.zip Dataset


Mounted at /content/drive/
Converted Dataset/animal/Cheetah/4838389045_d5fc08ca2b_b.jpg to RGB
Converted Dataset/scenery/desert/52712003999_cba6904a86_b.jpg to RGB
Converted Dataset/scenery/volcano/49629211101_d46f735055.jpg to RGB
  adding: Dataset/ (stored 0%)
  adding: Dataset/animal/ (stored 0%)
  adding: Dataset/animal/horse/ (stored 0%)
  adding: Dataset/animal/horse/38183707732_f4ca7d3b43_b.jpg (deflated 0%)
  adding: Dataset/animal/horse/5412377706_d77700fc38_b.jpg (deflated 1%)
  adding: Dataset/animal/Bee/ (stored 0%)
  adding: Dataset/animal/Bee/14142817779_568406cd86_b.jpg (deflated 0%)
  adding: Dataset/animal/Bee/23809746334_43928c099f_b.jpg (deflated 0%)
  adding: Dataset/animal/bird/ (stored 0%)
  adding: Dataset/animal/bird/21709064798_67e964516d_b.jpg (deflated 0%)
  adding: Dataset/animal/bird/28761726521_6aee5e6315_b.jpg (deflated 0%)
  adding: Dataset/animal/Snail/ (stored 0%)
  adding: Dataset/animal/Snail/6662983427_4f10f9687a_b.jpg (deflated 0%)
  adding: Datase

In [7]:
# Download the file using gdown
!gdown --id 1--6fe48D9ydnTpLV1GKKqJ0pqpOXB3z_

# Unzip the downloaded file
!unzip Clean_Dataset.zip

# Import necessary modules
import os
import shutil
from collections import defaultdict

# Define the source and target directories
source_dir = "Dataset"
train_dir = "data/train"
test_dir = "data/test"

# Create the target directories if they don’t exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Initialize a dictionary to hold file paths for each class
class_files = defaultdict(list)

# Read the file paths from the text file
with open('filename.txt', 'r') as file:
    lines = file.readlines()
    for line in lines:
        line = line.strip()
        if line:
            # Extract the class name from the path
            parts = line.split('/')
            class_name = parts[2]  # Structure: Dataset/category/class/image.jpg
            class_files[class_name].append(line)

# Move images to the train and test directories
for class_name, files in class_files.items():
    # Create the train and test directories for the class
    train_class_dir = os.path.join(train_dir, class_name)
    test_class_dir = os.path.join(test_dir, class_name)
    os.makedirs(train_class_dir, exist_ok=True)
    os.makedirs(test_class_dir, exist_ok=True)

    # Move 19 images to train and 1 image to test
    for i, file_path in enumerate(files):
        if i == 0:
            shutil.copy(file_path, test_class_dir)
        elif i < 20:
            shutil.copy(file_path, train_class_dir)

print("Dataset organization complete!")


/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1--6fe48D9ydnTpLV1GKKqJ0pqpOXB3z_
From (redirected): https://drive.google.com/uc?id=1--6fe48D9ydnTpLV1GKKqJ0pqpOXB3z_&confirm=t&uuid=bbbf8f2d-7329-4239-a56a-41280e1ed964
To: /content/Clean_Dataset.zip
100% 483M/483M [00:14<00:00, 33.9MB/s]
Archive:  Clean_Dataset.zip
replace Dataset/plant/Clove/53742600482_be65dab11c_b.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: Dataset/plant/Clove/53742600482_be65dab11c_b.jpg  
  inflating: Dataset/plant/Clove/6499672315_313ec22f51_b.jpg  
  inflating: Dataset/plant/Clove/22440611562_e293ed2cb6_b.jpg  
  inflating: Dataset/plant/Clove/46819415374_03d9b1b5b4_b.jpg  
  inflating: Dataset/plant/Clove/16282799783_cd1a50bdb9_b.jpg  
  inflating: Dataset/plant/Clov